# Homework 3
## Rodrigo Rivera

1. Estimate the PageRank for the given Web (see the Graph) with finite number of iterations using your own developed program:
    
    With no taxation
    
    Β=0.9
2. Define the criteria of stopping (e.g. ε<0.001)
3. Report should contain:
    The structure of the Web (Graph)
    Description of code / script and etc.
4. Conclusions

In [2]:
from array import array
from ctypes import c_double,pointer
import fileinput
import sys
import time
import os

In [4]:
def readGraph(lines):
    g = {} # node number -> out-link counts 
    inG = {} # node number -> set of in-link node numbers
    nodes = set() # all nodes
    for line in lines:
        #line = line.strip()
        if line.startswith('#'):
            continue
        l = line.split()
        k = int(l[0])
        v = int(l[-1])
        g.setdefault(k, set()).add(v)
        #inG.setdefault(v, set()).add(k)
        nodes.add(k)
        nodes.add(v)
    return g, inG, sorted(nodes)

In [9]:
def diff_vector(l1, l2):
    s = 0.0
    size = len(l1)
    for i in range(0,size-1):
        s += abs(l1[i] - l2[i])
    return s

In [6]:
def transformGraph(nodes, inG, g):
    idxInG = {} # instead of storing node number, store index of node -> index of in-nodexs
    idxG = {}
    idx = dict(zip(nodes, range(len(nodes))))
    for node, inNodes in inG.iteritems():
        idxInG[idx[node]] = set([idx[inNode] for inNode in inNodes])
    for node, outNodes in g.iteritems():
        idxG[idx[node]] = set([idx[outNode] for outNode in outNodes])
    return idxInG, idxG

In [11]:
def transformToArray(nodes, inG, g):
    a = array('l')
    idx = dict(zip(nodes, range(len(nodes))))
    idxInG, idxG = transformGraph(nodes, inG, g)
    size = len(nodes)
    for i in range(0, size-1): #[<index>, <size>, <out node list>]
        if i in idxG:
            a.append(i) 
            outNodes = idxG[i]
            a.append(len(outNodes))
            a.extend(outNodes)
    return idxInG, a

In [8]:
def fillVector(r, num):
    size = len(r)
    for i in range(0,size):
        r[i] = num

In [12]:
def forEachElement(r, func):
    size = len(r)
    for i in range(0, size-1):
        r[i] = func(r[i], i)

In [15]:
def pageRank(nodes, g, inG, beta=0.9, e=1e-8, n=1e2):
    r = array('d',[1.0/len(nodes)] * len(nodes))
    r_ = array('d', r)
    size = len(nodes)
    gSize = len(g)
    diff = e + 1
    t = 1
    while diff > e and (n == -1 or t < n):
        sum_r = 0.0
        # fill in 0s
        fillVector(r_, 0.0)
        # start iteration
        cursor = 0
        while cursor < gSize:
            i = g[cursor]
            cursor += 1
            row_size = g[cursor]
            cursor += 1
            row_end = cursor + row_size
            score_share = r[i] / row_size
            sum_r += r[i]
            while cursor < row_end :
                r_[ g[cursor] ] += score_share
                cursor += 1
    
    # multiply by beta
    sum_r = sum_r * beta
    leaked = 1 - sum_r
    leaked_share = leaked / size if leaked > 0 else 0.0
    
    _i = 0
    while _i < size:
        # multiply by beta and fix leak at the same time
        r_[_i] = r_[_i] * beta + leaked_share
        _i +=1
    
    diff = diff_vector(r, r_)
    r, r_ = r_, r
    t += 1
    
    return r, t

In [33]:
print >> sys.stderr, 'PID {0}'.format(os.getpid())
timer = time.time()


dir_path = 'workspace/hse_amda/'
full_file_path = 'web.txt'

f = open(full_file_path,'r')
for line in f.readlines():
    lines = line
    print line
f.close()

lines

# Directed graph (each unordered pair of nodes is saved once): web-Google.txt 

# Webgraph from the Google programming contest, 2002

# FromNodeId	ToNodeId

0 0

0 1

1 1

1 2

2 2

2 3

3 3

3 4

4 4

4 1


PID 1348


'4 1'

In [34]:
for line in lines:
    if line.startswith('#'):
        continue
    l = line.split()
    newline = line.split()
    print(newline)

['4']
[]
['1']


In [37]:
f = open(full_file_path,'r')
line2 = ''
for line in f.read().split('\n'):
    line2 = line
    print(line2)
f.close()
line2

# Directed graph (each unordered pair of nodes is saved once): web-Google.txt 
# Webgraph from the Google programming contest, 2002
# FromNodeId	ToNodeId
0 0
0 1
1 1
1 2
2 2
2 3
3 3
3 4
4 4
4 1


'4 1'

In [38]:
f

<closed file 'web.txt', mode 'r' at 0x7f2f49c79db0>

In [28]:


g, inG, nodes = readGraph(line2)
print >> sys.stderr, 'Read graph: {0:.1f} seconds.'.format(time.time() - timer)

timer = 0 - time.time()
inG, g = transformToArray(nodes, inG, g)
timer += time.time()
print >> sys.stderr, 'Preprocess graph: {0:.1f} seconds'.format(timer)

timer = 0 - time.time() 
r, t = pageRank(nodes, g, inG)
timer += time.time()
print >> sys.stderr, 'Page Rank: {0:.1f} seconds.'.format(timer)

for idx, i in enumerate(r):
    print '{0} {1:.10e}'.format(nodes[idx], i)
print >> sys.stderr, 'Done after {0} iterations.'.format(t)

IndexError: list index out of range